In [22]:
#Add repo path to the system path
from pathlib import Path
import os, sys
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
device = 0

from tqdm import tqdm
import SimpleITK as sitk
import numpy as np
from scipy.ndimage import label, generate_binary_structure

In [23]:
def lcc(mask:np.array):
    """generate largest connected component of a mask

    Args:
        mask (np.array): multi object mask

    Returns:
        np.array: array containing only the largest connected component
    """
    # Define the structuring element for connected component analysis
    structuring_element = generate_binary_structure(3, 1)  # 3x3x3 connectivity

    # Perform connected component labeling
    labeled_mask, _ = label(mask, structure=structuring_element)

    # Find the size of each connected component
    component_sizes = np.bincount(labeled_mask.ravel())

    # Identify the label of the largest component (excluding background)
    largest_component_label = np.argmax(component_sizes[1:]) + 1

    # Create a new mask containing only the largest component
    largest_component_mask = labeled_mask == largest_component_label
    # transform boolean to int
    largest_component_mask = largest_component_mask.astype(np.uint8)

    return largest_component_mask

In [19]:
# HP
high_threshold = 0.7

In [20]:
# load seed
seeds_dir = repo_path / 'experiments/inference/segmentation/data/predictions/full-size/high-probs_no-lcc_limed_0.98_top-hat_0.0001'
probs_dir = repo_path / 'experiments/inference/segmentation/data/predictions/full-size/multi-model_probs'
files = sorted(os.listdir(seeds_dir))

In [21]:
# for name in tqdm(files):
name = files[0]
# load probs
seed_path = seeds_dir / name
seed = sitk.ReadImage(str(seed_path))
seed = sitk.GetArrayFromImage(seed)

# get probability map
probs_path = probs_dir / name
probs = sitk.ReadImage(str(probs_path))
probs = sitk.GetArrayFromImage(probs)

In [31]:
# use high_threshold on prob map
possible_pixels = probs>high_threshold
possible_pixels = possible_pixels.astype(np.uint8)

# Define the structuring element for connected component analysis
structuring_element = generate_binary_structure(3, 1)  # 3x3x3 connectivity

# Perform connected component labeling
labeled_mask, _ = label(possible_pixels, structure=structuring_element)

intersection = labeled_mask*seed
ideal_group = np.max(intersection)

# use ideal group
mask = labeled_mask == ideal_group
mask = mask.astype(np.uint8)
